In [1]:
from pyspark.sql import SparkSession, SQLContext

import sparknlp
# required imports.  It's probably better to import specific classes rather than *
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
from sparknlp.pretrained import PretrainedPipeline

spark = (SparkSession.builder
         .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.11:2.4.0")
         .config("spark.driver.memory","16g") 
         .getOrCreate())

sc = spark.sparkContext
print(type(sc))
sqlContext = SQLContext(sc)
print(type(sqlContext))

print(sparknlp.version())

spark

<class 'pyspark.context.SparkContext'>
<class 'pyspark.sql.context.SQLContext'>
2.4.0


In [ ]:
VER = 'addvec'

In [2]:
# load wrangled data
raw_sdf = spark.read.json('./sample4.json')
raw_sdf.printSchema()

root
 |-- authors: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- date_download: string (nullable = true)
 |-- date_modify: string (nullable = true)
 |-- date_publish: string (nullable = true)
 |-- description: string (nullable = true)
 |-- language: string (nullable = true)
 |-- language_guess: string (nullable = true)
 |-- month: long (nullable = true)
 |-- published: string (nullable = true)
 |-- source_domain: string (nullable = true)
 |-- text: string (nullable = true)
 |-- text_or_desc: string (nullable = true)
 |-- title: string (nullable = true)
 |-- url: string (nullable = true)
 |-- year: long (nullable = true)



In [3]:
raw_sdf.take(2)

[Row(authors=[], date_download='2019-11-27 22:51:37', date_modify='2019-11-27 22:51:37', date_publish='2015-03-27 17:17:42', description="Federal Reserve Chairman Ben Bernanke will begin holding news conferences four times a year to answer questions about the Federal Reserve's policy decisions. It represents a significant shift in...", language='en', language_guess='en', month=3, published='2015-03-27T17:17:42.000Z', source_domain='foxnews.com', text='Federal Reserve Chairman Ben Bernanke will begin holding news conferences four times a year to answer questions about the Federal Reserve\'s policy decisions. It represents a significant shift in strategy for the central bank, one that will give it a chance to defend actions that in recent months have faced harsh criticism.\nThe decision, which was announced Thursday, comes after the Fed held an unusual videoconference last fall in large part to discuss the need to improve its communications with the public. A Fed committee also had been 

In [4]:
import pyspark.sql.functions as F
from pyspark.sql.functions import when, col

In [5]:
# add id column
processed_sdf = raw_sdf.select("*").withColumn("id", F.monotonically_increasing_id())

In [6]:
import contractions
contractions_udf = F.udf(contractions.fix)
# simple fix to contractions
processed_sdf = processed_sdf.withColumn('clean_text', contractions_udf('text_or_desc'))

In [7]:
processed_sdf.select('clean_text').take(2)

[Row(clean_text='Federal Reserve Chairman Ben Bernanke will begin holding news conferences four times a year to answer questions about the Federal Reserve\'s policy decisions. It represents a significant shift in strategy for the central bank, one that will give it a chance to defend actions that in recent months have faced harsh criticism.\nThe decision, which was announced Thursday, comes after the Fed held an unusual videoconference last fall in large part to discuss the need to improve its communications with the public. A Fed committee also had been studying whether to begin holding periodic news conferences.\nBernanke\'s first news conference will take place after the Fed\'s April 27 meeting. That will augment the current communications strategy: a brief statement released after each of the Fed\'s eight policy-making meetings with no officials available to answer questions.\nit is a notable change for the Fed. Its chairmen rarely take questions from reporters, and when they do, t

In [8]:
# remove possessives
processed_sdf = processed_sdf.withColumn('clean_text', F.regexp_replace('clean_text', "'s", ""))

In [9]:
processed_sdf.select("clean_text").take(2)

[Row(clean_text='Federal Reserve Chairman Ben Bernanke will begin holding news conferences four times a year to answer questions about the Federal Reserve policy decisions. It represents a significant shift in strategy for the central bank, one that will give it a chance to defend actions that in recent months have faced harsh criticism.\nThe decision, which was announced Thursday, comes after the Fed held an unusual videoconference last fall in large part to discuss the need to improve its communications with the public. A Fed committee also had been studying whether to begin holding periodic news conferences.\nBernanke first news conference will take place after the Fed April 27 meeting. That will augment the current communications strategy: a brief statement released after each of the Fed eight policy-making meetings with no officials available to answer questions.\nit is a notable change for the Fed. Its chairmen rarely take questions from reporters, and when they do, they are ofte

In [10]:
# remove non text 
processed_sdf = processed_sdf.withColumn('clean_text', F.regexp_replace('clean_text',"[^A-Za-z\. \n]", ""))

In [11]:
# standardize US
processed_sdf = processed_sdf.withColumn('clean_text', F.regexp_replace('clean_text',"(US|U.S.|USA)", "United States"))

In [12]:
processed_sdf.select("clean_text").take(2)

[Row(clean_text='Federal Reserve Chairman Ben Bernanke will begin holding news conferences four times a year to answer questions about the Federal Reserve policy decisions. It represents a significant shift in strategy for the central bank one that will give it a chance to defend actions that in recent months have faced harsh criticism.\nThe decision which was announced Thursday comes after the Fed held an unusual videoconference last fall in large part to discuss the need to improve its communications with the public. A Fed committee also had been studying whether to begin holding periodic news conferences.\nBernanke first news conference will take place after the Fed April  meeting. That will augment the current communications strategy a brief statement released after each of the Fed eight policymaking meetings with no officials available to answer questions.\nit is a notable change for the Fed. Its chairmen rarely take questions from reporters and when they do they are often guarded

In [13]:
documentAssembler = DocumentAssembler().setInputCol("clean_text").setOutputCol("document")

In [14]:
sentenceDetector = SentenceDetector().setInputCols(["document"]).setOutputCol("sentence")

In [15]:
tokenizer = (Tokenizer() 
    .setInputCols(["sentence"]) 
    .setOutputCol("token") 
            )

In [16]:
lemma = (LemmatizerModel.pretrained('lemma_antbnc')
        .setInputCols(['token'])
         .setOutputCol('lemma')
        )

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]


In [17]:
stop_words_cleaner = (StopWordsCleaner()
                      .setInputCols(['lemma'])
                      .setOutputCol('stopless')
                     )
stop_words = stop_words_cleaner.getStopWords()
stop_words.remove('not')
stop_words.remove('cannot')
stop_words.remove('against')
stop_words.remove('nor')
stop_words.remove('no')
stop_words = [word.replace("'", "") for word in stop_words]
stop_words_cleaner.setStopWords(stop_words)
stop_words_cleaner.getStopWords()

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 'her',
 'hers',
 'herself',
 'it',
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each',
 'few',
 'more',
 'most',
 'other',
 'some',
 'such',
 'only',
 'own',
 'same',
 's

In [18]:
normalizer = (Normalizer() \
    .setInputCols(["stopless"]) \
    .setOutputCol("normalized")
             )

In [19]:
word_embed = (WordEmbeddingsModel.pretrained()
             .setInputCols(['sentence', 'stopless'])
             .setOutputCol('embedding')
             )

glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]


In [20]:
ner_dl = (NerDLModel().pretrained()
          .setInputCols(['document', 'stopless', 'embedding'])
          .setOutputCol('ner')
         )

ner_dl download started this may take some time.
Approximate size to download 13.5 MB
[OK!]


In [21]:
ner_conv = (NerConverter()
            .setInputCols(["document", "normalized", "ner"])
            .setOutputCol('ner_converted')
           )
print(ner_conv.explainParams())

inputCols: previous annotations columns, if renamed (current: ['document', 'normalized', 'ner'])
lazyAnnotator: Whether this AnnotatorModel acts as lazy in RecursivePipelines (default: False)
outputCol: output annotation column. can be left default. (current: ner_converted)
whiteList: If defined, list of entities to process. The rest will be ignored. Do not include IOB prefix on labels (undefined)


In [22]:
sentiment_model = (ViveknSentimentModel.pretrained()
                   .setInputCols(['document', 'token'])
                   .setOutputCol('sentiment')
                  )

sentiment_vivekn download started this may take some time.
Approximate size to download 873.6 KB
[OK!]


In [23]:
finisher = Finisher().setInputCols(["sentence", "token", "lemma", "stopless", "embedding", "normalized", "ner", "ner_converted", 'sentiment']).setCleanAnnotations(True)

In [24]:
pipeline = (Pipeline()
    .setStages([
        documentAssembler,
        sentenceDetector,
        tokenizer,
        lemma,
        stop_words_cleaner,
        normalizer,
        word_embed,
        ner_dl,
        ner_conv,
        sentiment_model,
        finisher
    ]))

In [25]:
pipeline_model = pipeline.fit(processed_sdf)

In [26]:
processed_sdf = pipeline_model.transform(processed_sdf)

In [27]:
processed_sdf.printSchema()

root
 |-- authors: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- date_download: string (nullable = true)
 |-- date_modify: string (nullable = true)
 |-- date_publish: string (nullable = true)
 |-- description: string (nullable = true)
 |-- language: string (nullable = true)
 |-- language_guess: string (nullable = true)
 |-- month: long (nullable = true)
 |-- published: string (nullable = true)
 |-- source_domain: string (nullable = true)
 |-- text: string (nullable = true)
 |-- text_or_desc: string (nullable = true)
 |-- title: string (nullable = true)
 |-- url: string (nullable = true)
 |-- year: long (nullable = true)
 |-- id: long (nullable = false)
 |-- clean_text: string (nullable = true)
 |-- finished_sentence: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- finished_token: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- finished_lemma: array (nullable = true)
 |    |-- element: string (conta

In [28]:
processed_sdf.select('clean_text', 'finished_sentiment').take(50)

[Row(clean_text='Federal Reserve Chairman Ben Bernanke will begin holding news conferences four times a year to answer questions about the Federal Reserve policy decisions. It represents a significant shift in strategy for the central bank one that will give it a chance to defend actions that in recent months have faced harsh criticism.\nThe decision which was announced Thursday comes after the Fed held an unusual videoconference last fall in large part to discuss the need to improve its communications with the public. A Fed committee also had been studying whether to begin holding periodic news conferences.\nBernanke first news conference will take place after the Fed April  meeting. That will augment the current communications strategy a brief statement released after each of the Fed eight policymaking meetings with no officials available to answer questions.\nit is a notable change for the Fed. Its chairmen rarely take questions from reporters and when they do they are often guarded

In [29]:
processed_sdf = processed_sdf.withColumn('sentiment',  
          when(col('finished_sentiment')[0] == 'positive', 1).when(col('finished_sentiment')[0] == 'negative', -1).otherwise(0)
                )

In [30]:
processed_sdf.select('finished_sentiment', 'sentiment').take(5)

[Row(finished_sentiment=['positive'], sentiment=1),
 Row(finished_sentiment=['negative'], sentiment=-1),
 Row(finished_sentiment=['positive'], sentiment=1),
 Row(finished_sentiment=['positive'], sentiment=1),
 Row(finished_sentiment=['negative'], sentiment=-1)]

In [1]:
processed_sdf.take(3)

NameError: name 'processed_sdf' is not defined

In [31]:
from pyspark.ml.feature import CountVectorizer, IDF

In [32]:
cv = CountVectorizer(inputCol="finished_ner_converted", 
                     outputCol="ner_vectors",
                     maxDF=0.8,
                     minDF=3
                    )

In [33]:
print(cv.explainParams())

binary: Binary toggle to control the output vector values. If True, all nonzero counts (after minTF filter applied) are set to 1. This is useful for discrete probabilistic models that model binary events rather than integer counts. Default False (default: False)
inputCol: input column name. (current: finished_ner_converted)
maxDF: Specifies the maximum number of different documents a term could appear in to be included in the vocabulary. A term that appears more than the threshold will be ignored. If this is an integer >= 1, this specifies the maximum number of documents the term could appear in; if this is a double in [0,1), then this specifies the maximum fraction of documents the term could appear in. Default (2^63) - 1 (default: 9.223372036854776e+18, current: 0.8)
minDF: Specifies the minimum number of different documents a term must appear in to be included in the vocabulary. If this is an integer >= 1, this specifies the number of documents the term must appear in; if this is a 

In [34]:
cv.getVocabSize()

262144

In [35]:
cv_model = cv.fit(processed_sdf)

In [36]:
processed_sdf = cv_model.transform(processed_sdf)

In [37]:
processed_sdf.printSchema()

root
 |-- authors: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- date_download: string (nullable = true)
 |-- date_modify: string (nullable = true)
 |-- date_publish: string (nullable = true)
 |-- description: string (nullable = true)
 |-- language: string (nullable = true)
 |-- language_guess: string (nullable = true)
 |-- month: long (nullable = true)
 |-- published: string (nullable = true)
 |-- source_domain: string (nullable = true)
 |-- text: string (nullable = true)
 |-- text_or_desc: string (nullable = true)
 |-- title: string (nullable = true)
 |-- url: string (nullable = true)
 |-- year: long (nullable = true)
 |-- id: long (nullable = false)
 |-- clean_text: string (nullable = true)
 |-- finished_sentence: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- finished_token: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- finished_lemma: array (nullable = true)
 |    |-- element: string (conta

In [38]:
processed_sdf.take(1)

[Row(authors=[], date_download='2019-11-27 22:51:37', date_modify='2019-11-27 22:51:37', date_publish='2015-03-27 17:17:42', description="Federal Reserve Chairman Ben Bernanke will begin holding news conferences four times a year to answer questions about the Federal Reserve's policy decisions. It represents a significant shift in...", language='en', language_guess='en', month=3, published='2015-03-27T17:17:42.000Z', source_domain='foxnews.com', text='Federal Reserve Chairman Ben Bernanke will begin holding news conferences four times a year to answer questions about the Federal Reserve\'s policy decisions. It represents a significant shift in strategy for the central bank, one that will give it a chance to defend actions that in recent months have faced harsh criticism.\nThe decision, which was announced Thursday, comes after the Fed held an unusual videoconference last fall in large part to discuss the need to improve its communications with the public. A Fed committee also had been 

In [39]:
cv_model.vocabulary

['Trump',
 'United States',
 'Obama',
 'American',
 'Democrats',
 'Americans',
 'Congress',
 'Republican',
 'Bush',
 'Republicans',
 'America',
 'Democratic',
 'White House',
 'Russian',
 'Senate',
 'Russia',
 'Iraq',
 'Washington',
 'China',
 'Israel',
 'Clinton',
 'New York',
 'GOP',
 'Chinese',
 'Iran',
 'McCain',
 'Donald Trump',
 'House',
 'Hillary',
 'British',
 'Trumps',
 'Israeli',
 'FBI',
 'Facebook',
 'God',
 'French',
 'Privacy Policy',
 'Supreme Court',
 'Syria',
 'Nations',
 'European',
 'Europe',
 'Texas',
 'California',
 'Take Action',
 'Chip',
 'Nation Travels',
 'Ukraine',
 'Iraqi',
 'EU',
 'Chicago',
 'Houston',
 'Romney',
 'Florida',
 'Google',
 'AP',
 'Barack Obama',
 'Syrian',
 'Afghanistan',
 'New York Times',
 'Jewish',
 'Reuters',
 'Muslim',
 'Moscow',
 'CNN',
 'German',
 'NPR',
 'Edwards',
 'Palestinian',
 'Americas',
 'Japanese',
 'India',
 'Internet',
 'Iowa',
 'UK',
 'Medicare',
 'English',
 'Britain',
 'Western',
 'Putin',
 'Iranian',
 'NFL',
 'Johnson',
 '

In [40]:
cv.getVocabSize()

262144

In [41]:
idf = IDF(inputCol="ner_vectors", outputCol="ner_vectors_idf")

In [42]:
idfModel = idf.fit(processed_sdf)

In [43]:
processed_sdf = idfModel.transform(processed_sdf)

In [44]:
processed_sdf.take(1)

[Row(authors=[], date_download='2019-11-27 22:51:37', date_modify='2019-11-27 22:51:37', date_publish='2015-03-27 17:17:42', description="Federal Reserve Chairman Ben Bernanke will begin holding news conferences four times a year to answer questions about the Federal Reserve's policy decisions. It represents a significant shift in...", language='en', language_guess='en', month=3, published='2015-03-27T17:17:42.000Z', source_domain='foxnews.com', text='Federal Reserve Chairman Ben Bernanke will begin holding news conferences four times a year to answer questions about the Federal Reserve\'s policy decisions. It represents a significant shift in strategy for the central bank, one that will give it a chance to defend actions that in recent months have faced harsh criticism.\nThe decision, which was announced Thursday, comes after the Fed held an unusual videoconference last fall in large part to discuss the need to improve its communications with the public. A Fed committee also had been 

In [45]:
ner_vectors = processed_sdf.select('ner_vectors')
sv_size = ner_vectors.first()[0].size
sv_size

7379

In [46]:
ner_vectors_idf = processed_sdf.select('ner_vectors_idf')

In [47]:
data_size = processed_sdf.count()
data_size

10000

In [48]:
import numpy as np
sum = np.zeros(sv_size)


In [49]:
sum_scale = np.zeros(sv_size)

In [50]:
for row in ner_vectors.take(data_size):
    sum += row[0].toArray()

In [51]:
for row in ner_vectors_idf.take(data_size):
    sum_scale += row[0].toArray()

In [52]:
sum[0:20]

array([2799., 2630., 2552., 2002., 1224., 1045., 1042., 1011., 1001.,
        969.,  962.,  896.,  879.,  848.,  840.,  833.,  766.,  731.,
        719.,  668.])

In [53]:
accumulate = []
for i,j in enumerate(sum):
    accumulate.append((i,j))

In [54]:
accumulate.sort(key=lambda x: -x[1])

In [55]:
accumulate_scale = []
for i,j in enumerate(sum_scale):
    accumulate_scale.append((i,j))
accumulate_scale.sort(key=lambda x: -x[1])

In [56]:
for i in range(len(accumulate)):
    item = accumulate[i]
    print(i+1, "|", cv_model.vocabulary[item[0]], "| ", item[1])


1 | Trump |  2799.0
2 | United States |  2630.0
3 | Obama |  2552.0
4 | American |  2002.0
5 | Democrats |  1224.0
6 | Americans |  1045.0
7 | Congress |  1042.0
8 | Republican |  1011.0
9 | Bush |  1001.0
10 | Republicans |  969.0
11 | America |  962.0
12 | Democratic |  896.0
13 | White House |  879.0
14 | Russian |  848.0
15 | Senate |  840.0
16 | Russia |  833.0
17 | Iraq |  766.0
18 | Washington |  731.0
19 | China |  719.0
20 | Israel |  668.0
21 | Clinton |  617.0
22 | New York |  597.0
23 | GOP |  581.0
24 | Chinese |  558.0
25 | Iran |  544.0
26 | McCain |  538.0
27 | Donald Trump |  527.0
28 | House |  511.0
29 | Hillary |  473.0
30 | British |  444.0
31 | Trumps |  431.0
32 | Israeli |  427.0
33 | FBI |  406.0
34 | Facebook |  403.0
35 | God |  396.0
36 | French |  395.0
37 | Privacy Policy |  395.0
38 | Supreme Court |  390.0
39 | Syria |  384.0
40 | Nations |  376.0
41 | European |  375.0
42 | Europe |  370.0
43 | Texas |  369.0
44 | California |  367.0
45 | Take Action | 

253 | Jets |  89.0
254 | Cuba |  89.0
255 | Arizona |  89.0
256 | Read Full Post |  88.0
257 | Visit |  88.0
258 | New Orleans |  88.0
259 | EBITDA |  88.0
260 | Ohio |  87.0
261 | Kremlin |  87.0
262 | Vladimir Putin |  87.0
263 | Asia |  87.0
264 | Swiss |  87.0
265 | Hong Kong |  87.0
266 | San Diego |  87.0
267 | Kennedy |  87.0
268 | Italy |  86.0
269 | Wisconsin |  86.0
270 | Olympics |  86.0
271 | South Carolina |  85.0
272 | AfricanAmerican |  85.0
273 | Britains |  85.0
274 | Amendment |  85.0
275 | Kavanaugh |  84.0
276 | Irish |  84.0
277 | Jordan |  84.0
278 | Soviet Union |  84.0
279 | Alaska |  83.0
280 | Israelis |  83.0
281 | Harvard |  83.0
282 | Starbucks |  83.0
283 | Slate |  83.0
284 | Latinos |  83.0
285 | Assad |  83.0
286 | Gore |  82.0
287 | Israels |  82.0
288 | Hispanic |  82.0
289 | Patriots |  82.0
290 | Tehran |  82.0
291 | Eagles |  81.0
292 | New Hampshire |  81.0
293 | District |  81.0
294 | Huawei |  81.0
295 | NASA |  81.0
296 | Spain |  81.0
297 | Ra

617 | IAEA |  41.0
618 | Cabinet |  41.0
619 | Epstein |  40.0
620 | Brady |  40.0
621 | African Americans |  40.0
622 | AFP |  40.0
623 | Court |  40.0
624 | Abbas |  40.0
625 | Rasmussen |  40.0
626 | Powell |  40.0
627 | Romneys |  40.0
628 | Bruins |  40.0
629 | Martinez |  40.0
630 | Macron |  39.0
631 | F |  39.0
632 | Brennan |  39.0
633 | Mattis |  39.0
634 | Graham |  39.0
635 | UNITED STATES |  39.0
636 | Evans |  39.0
637 | Magic |  39.0
638 | George W. Bush |  39.0
639 | Morgan |  39.0
640 | Astros |  39.0
641 | Mitch McConnell |  39.0
642 | Maduro |  38.0
643 | FIFA |  38.0
644 | PAC |  38.0
645 | alQaeda |  38.0
646 | Duncan |  38.0
647 | McCarthy |  38.0
648 | Newsday |  38.0
649 | Andrew |  38.0
650 | Redskins |  38.0
651 | Marines |  38.0
652 | Merkel |  38.0
653 | Colombia |  38.0
654 | Morris |  38.0
655 | Sweden |  38.0
656 | Ohio State |  38.0
657 | Lee |  38.0
658 | Puerto Rico |  37.0
659 | West Virginia |  37.0
660 | OConnor |  37.0
661 | Pruitt |  37.0
662 | Ho

983 | Ronaldo |  25.0
984 | Chelsea |  25.0
985 | Tampa Bay |  25.0
986 | Robert |  25.0
987 | Medicareforall |  25.0
988 | Marine |  25.0
989 | Booker |  25.0
990 | eBay |  25.0
991 | Iraq War |  25.0
992 | Jeb Bush |  25.0
993 | Secret Service |  25.0
994 | Bidens |  25.0
995 | Britons |  25.0
996 | Ted Cruz |  25.0
997 | Baker |  25.0
998 | Rhode Island |  25.0
999 | Iraq and Afghanistan |  25.0
1000 | Sunnis |  25.0
1001 | Scotland |  25.0
1002 | Peru |  25.0
1003 | Giffords |  25.0
1004 | New Republic |  25.0
1005 | Adams |  25.0
1006 | Tories |  25.0
1007 | Dylan |  25.0
1008 | South African |  25.0
1009 | Duffy |  25.0
1010 | Lakewood |  24.0
1011 | Ward |  24.0
1012 | Geneva |  24.0
1013 | NEW YORK |  24.0
1014 | Jazz |  24.0
1015 | Santorum |  24.0
1016 | Boehner |  24.0
1017 | Acosta |  24.0
1018 | Washington DC |  24.0
1019 | Leonard |  24.0
1020 | Saudis |  24.0
1021 | Karachi |  24.0
1022 | Chevron |  24.0
1023 | IG |  24.0
1024 | WhatsApp |  24.0
1025 | Labor Department |

1321 | Man |  19.0
1322 | Phillies |  19.0
1323 | Confederate |  19.0
1324 | CDC |  19.0
1325 | Nashville |  19.0
1326 | Five |  19.0
1327 | Juul |  19.0
1328 | Little |  19.0
1329 | United States of America |  19.0
1330 | Portugal |  19.0
1331 | Diaz |  19.0
1332 | ACA |  19.0
1333 | Greeks |  19.0
1334 | Wray |  19.0
1335 | Steve Jobs |  19.0
1336 | Morales |  19.0
1337 | Basra |  19.0
1338 | Conservative Party |  19.0
1339 | Shaw |  19.0
1340 | Aguilar |  19.0
1341 | Robert Muellers |  19.0
1342 | Minneapolis |  19.0
1343 | Financial Times |  19.0
1344 | Diallo |  19.0
1345 | Communist Party |  19.0
1346 | Garrett |  19.0
1347 | Ladens |  19.0
1348 | Scaife |  19.0
1349 | Kuwait |  19.0
1350 | Times Square |  19.0
1351 | West African |  19.0
1352 | Vietnamese |  19.0
1353 | Mark Zuckerberg |  19.0
1354 | Michael Brown |  19.0
1355 | Chuck Schumer |  19.0
1356 | Government |  19.0
1357 | Senegal |  19.0
1358 | SNP |  19.0
1359 | Palestinian Authority |  19.0
1360 | Sharpton |  19.0
1

...

6162 | Scarlett Johansson |  3.0
6163 | Emma Stone |  3.0
6164 | Kamath |  3.0
6165 | David Barrett |  3.0
6166 | Terry Branstad |  3.0
6167 | AWR Hawkins |  3.0
6168 | Pulitzer Prizewinning |  3.0
6169 | Tommy Thompson |  3.0
6170 | Peshmerga |  3.0
6171 | Ian Hanchett |  3.0
6172 | Freeport |  3.0
6173 | Beats |  3.0
6174 | CHICAGO AP |  3.0
6175 | Martha Coakley |  3.0
6176 | Adam Serwer |  3.0
6177 | Arianna Huffington |  3.0
6178 | PASADENA Calif |  3.0
6179 | Ryan Zinke |  3.0
6180 | Lexus |  3.0
6181 | Mignon Clyburn |  3.0
6182 | Lewis Carroll |  3.0
6183 | TMZ.co |  3.0
6184 | Democrat in the House |  3.0
6185 | Malcolm Gladwell |  3.0
6186 | Dearborn Michigan |  3.0
6187 | Zion Williamson |  3.0
6188 | Jim Armstrong |  3.0
6189 | Wall Streeters |  3.0
6190 | al Qaedalinked |  3.0
6191 | Following |  3.0
6192 | Marcos |  3.0
6193 | Besides |  3.0
6194 | Keynes |  3.0
6195 | Freddie |  3.0
6196 | Grahams |  3.0
6197 | NPR.or |  3.0
6198 | Chris Hayes |  3.0
6199 | Helen Regan |

6445 | Scott Gottlieb |  3.0
6446 | Anthony Weiner |  3.0
6447 | New York Stock Exchange NYSE |  3.0
6448 | Robert Kuttner |  3.0
6449 | Norman Mailer |  3.0
6450 | Ice |  3.0
6451 | Beatle |  3.0
6452 | Cali |  3.0
6453 | Aleksey Navalny |  3.0
6454 | Eric Shinseki |  3.0
6455 | Michael Graham |  3.0
6456 | Honey |  3.0
6457 | Kwame Brown |  3.0
6458 | Galen |  3.0
6459 | Dana |  3.0
6460 | Peterson Institute for International Economics |  3.0
6461 | HTML |  3.0
6462 | Trump in Senate |  3.0
6463 | David Denby |  3.0
6464 | CAIR |  3.0
6465 | John Kennedy |  3.0
6466 | Center for American Progress |  3.0
6467 | National Book Award |  3.0
6468 | Exxon Mobil Corp |  3.0
6469 | Paul Johnson |  3.0
6470 | George Springer |  3.0
6471 | Veterans Affairs Department |  3.0
6472 | Lisa Murkowski |  3.0
6473 | George H |  3.0
6474 | Securities Exchange Act |  3.0
6475 | Slate on Facebook |  3.0
6476 | Federal Government |  3.0
6477 | LaGuardia Airport |  3.0
6478 | Zionists |  3.0
6479 | Old So

6741 | Pat Toomey |  3.0
6742 | PTA |  3.0
6743 | Puritan |  3.0
6744 | State Senate |  3.0
6745 | Eastern Washington |  3.0
6746 | Go |  3.0
6747 | Joe the Plumber |  3.0
6748 | Moroccan |  3.0
6749 | Conan OBrien |  3.0
6750 | Best Defense |  3.0
6751 | Florida Lotterys Pick |  3.0
6752 | Obamacares |  3.0
6753 | WTI |  3.0
6754 | Fitbit |  3.0
6755 | Upton |  3.0
6756 | Boston Celtics |  3.0
6757 | Peter Dawson |  3.0
6758 | Independence Day |  3.0
6759 | Wise |  3.0
6760 | Yitzhak Rabin |  3.0
6761 | Fray |  3.0
6762 | Vermonts |  3.0
6763 | Ban KiMoon |  3.0
6764 | Carlos Carrasco |  3.0
6765 | Libertarians |  3.0
6766 | Air Raid |  3.0
6767 | Nick Cleggs |  3.0
6768 | Khabibs |  3.0
6769 | Tesla Model |  3.0
6770 | John F. Kerry |  3.0
6771 | Bernies |  3.0
6772 | Courtesy |  3.0
6773 | Blink |  3.0
6774 | Concord |  3.0
6775 | Rocky |  3.0
6776 | Albert Almora Jr |  3.0
6777 | TEXAS UNITED STATES |  3.0
6778 | Oakland Athletics |  3.0
6779 | Chamberlain |  3.0
6780 | Clint Eastw

7034 | Pedro Sanchez |  3.0
7035 | Foundation for Defense of Democracies |  3.0
7036 | Black Panthers |  3.0
7037 | Merrick |  3.0
7038 | Housing and Urban Development |  3.0
7039 | Copyright Business Wire .
PUB   PMDISC |  3.0
7040 | Plain |  3.0
7041 | nonCatholic |  3.0
7042 | Midshipmen |  3.0
7043 | Joanne Ostrow |  3.0
7044 | HBOs |  3.0
7045 | Country |  3.0
7046 | Rafik Hariri |  3.0
7047 | Ankaras |  3.0
7048 | Downton Abbey |  3.0
7049 | Scotland Yard |  3.0
7050 | Arabs and Muslims |  3.0
7051 | Lone Star State |  3.0
7052 | Moscow Region |  3.0
7053 | Diane Sawyer |  3.0
7054 | Parkland Florida |  3.0
7055 | Big Oil |  3.0
7056 | Bachelor |  3.0
7057 | Pepsi |  3.0
7058 | Crime Stoppers |  3.0
7059 | Englands |  3.0
7060 | App Store |  3.0
7061 | alNusra Front |  3.0
7062 | Central Valley |  3.0
7063 | Marx |  3.0
7064 | Catalan |  3.0
7065 | Jason Campbell |  3.0
7066 | Legislatures |  3.0
7067 | Super Bowl MVP |  3.0
7068 | Jinpings |  3.0
7069 | Henry Hazlitt |  3.0
7070

7328 | Miamis |  3.0
7329 | Bono |  3.0
7330 | Clint Capela |  3.0
7331 | notat |  3.0
7332 | Carolina Panthers |  3.0
7333 | Adam Silver |  3.0
7334 | Qaeda in Iraq |  3.0
7335 | DENVER |  3.0
7336 | Free Democrats |  3.0
7337 | Daniel Patrick Moynihan |  3.0
7338 | Justin Rose |  3.0
7339 | BMI |  3.0
7340 | Chinayou. |  3.0
7341 | Boy Scout |  3.0
7342 | Chuck Grassley |  3.0
7343 | Israeli Jews |  3.0
7344 | Rep. Ron Paul |  3.0
7345 | Indepth |  3.0
7346 | antiIslamic |  3.0
7347 | Charlize Theron |  3.0
7348 | Javy Baez |  3.0
7349 | Budapest |  3.0
7350 | Rebecca Hennes |  3.0
7351 | National Oceanic and Atmospheric Administration |  3.0
7352 | Patti |  3.0
7353 | Hubert Humphrey |  3.0
7354 | Hudson Valley |  3.0
7355 | People on the Move |  3.0
7356 | Mike Brown |  3.0
7357 | Gingrichs |  3.0
7358 | Henry Waxman |  3.0
7359 | Public Religion Research Institute |  3.0
7360 | Volcker |  3.0
7361 | Sandra Maler |  3.0
7362 | Bear |  3.0
7363 | Ellen Ladowsky |  3.0
7364 | Nativit

In [57]:
processed_sdf.count()

10000

In [58]:
for i in range(len(accumulate_scale)):
    item = accumulate_scale[i]
    print(i+1, "|", cv_model.vocabulary[item[0]], "| ", item[1])


1 | Trump |  7380.291799844406
2 | Obama |  6801.311483350598
3 | United States |  5627.45376305372
4 | American |  4401.046795485617
5 | Democrats |  3654.719491746246
6 | Bush |  3396.7225362203126
7 | Congress |  3016.6758248032265
8 | Republican |  2989.1342825104593
9 | Republicans |  2973.282469586091
10 | Americans |  2919.4249482650716
11 | Russia |  2867.2854361952395
12 | Russian |  2847.785728347319
13 | Iraq |  2792.774689094833
14 | Democratic |  2792.7160548505085
15 | America |  2722.765752533795
16 | Senate |  2666.965504939473
17 | White House |  2649.302694240963
18 | China |  2602.3869848688078
19 | Israel |  2600.0700092523803
20 | McCain |  2546.409711244091
21 | Clinton |  2366.2949288833875
22 | Hillary |  2353.8059169053895
23 | Washington |  2219.7942663074978
24 | Iran |  2200.8359898208323
25 | Chinese |  2174.6567884675806
26 | GOP |  2035.4347893942643
27 | New York |  1941.4533573462045
28 | Israeli |  1813.723379577993
29 | House |  1786.8175639097192
30 

235 | Huawei |  499.43935226950066
236 | Michigan |  497.8141030753176
237 | Donald Trumps |  497.36917954675033
238 | Greek |  496.0693535176289
239 | San Francisco |  494.1881775974614
240 | NCAA |  493.14693440451424
241 | Hong Kong |  491.9930305773778
242 | Asian |  491.23256736778916
243 | Swiss |  487.1584535269095
244 | Kennedy |  482.57844871367746
245 | Dallas |  482.4928747686546
246 | Treasury |  481.9954199952082
247 | New Orleans |  481.6038258849971
248 | Capitol Hill Publishing Corp |  480.98682244654947
249 | News Communications Inc |  480.98682244654947
250 | Hill  K Street NW Suite  Washington DC |  480.98682244654947
251 | Rangers |  479.3251633686653
252 | Snowden |  478.5325231935345
253 | Flynn |  477.5128939203846
254 | Pelosi |  471.55914517780514
255 | San Diego |  470.1286753169395
256 | Instagram |  469.79482737636613
257 | Thompson |  468.6097626232351
258 | Miami |  468.3529380285213
259 | Assad |  464.76036370958036
260 | Olympics |  462.83471146191795
26

396 | Kansas |  360.7908135593035
397 | North |  360.7908135593035
398 | Federal Reserve |  360.2804861084448
399 | Politico |  358.9458019291488
400 | Kosovo |  357.6314591230869
401 | Bradley |  357.6232398966795
402 | NYPD |  357.16207910517426
403 | Washington D. |  355.8314187351997
404 | Cold War |  353.6671871948966
405 | Kurdish |  353.3798984232052
406 | Disney |  352.6941514080789
407 | Indians |  352.41295022634176
408 | Eddie |  352.08657026354814
409 | Ebola |  351.4573219674262
410 | Philadelphia |  351.38245874165386
411 | FDA |  351.188285908347
412 | Northam |  350.6607894820148
413 | Qaeda |  350.61572294020027
414 | NPRs |  350.2979716051925
415 | Verbtm Inc. an NPR |  350.2979716051925
416 | Walker |  349.878419124457
417 | Blair |  347.4397998030854
418 | Silicon Valley |  347.1703921686021
419 | Bell |  345.5655324947789
420 | Howard |  345.2914040381732
421 | Seattle |  343.05873697705334
422 | Kim |  343.047003056387
423 | New England |  341.57086971426986
424 |

628 | New Yorkers |  247.0222919596019
629 | Cain |  246.9940274093209
630 | Rasmussen |  246.63671717012363
631 | Syracuse |  245.88316735205422
632 | Broadway |  245.69911063693166
633 | Latin America |  245.41207055358183
634 | TSA |  245.25162339041327
635 | McCabe |  244.8164696256992
636 | Court |  244.775916544728
637 | Redskins |  244.6383624999959
638 | Macron |  244.37405412459293
639 | North America |  242.94868016595362
640 | State |  242.9486801659536
641 | HBO |  242.6110328246376
642 | Cabinet |  242.49874353985973
643 | Sarah Palin |  241.95762142637741
644 | Huskies |  241.77493458938648
645 | Watson |  241.77493458938648
646 | Senate Majority |  240.80191294249835
647 | Sandy |  240.5884361373494
648 | Andrew |  240.16260714505336
649 | Merkel |  240.16260714505336
650 | Ohio State |  240.16260714505336
651 | Midwest |  239.9851778314216
652 | Einstein |  239.72949719139967
653 | Hernandez |  239.23767634253926
654 | Guardian |  237.5138768304583
655 | Nancy |  237.39

856 | Pistons |  185.40596945771367
857 | Ben |  185.40596945771364
858 | Gods |  184.52265869760612
859 | McDonald |  183.9987250461152
860 | IDF |  183.99872504611517
861 | Spurs |  183.99872504611517
862 | Koch |  183.99872504611517
863 | Apollo |  183.99872504611514
864 | Johnsons |  183.99872504611514
865 | Goldman |  183.93871754280994
866 | Montreal |  183.581937408546
867 | Theresa |  183.581937408546
868 | North Dakota |  183.581937408546
869 | Latin |  183.35270853013787
870 | Page  Buy Reprints |  182.84584977632122
871 | Lakewood |  182.424058909018
872 | Roy |  182.424058909018
873 | Edison |  182.424058909018
874 | Chevron |  182.42405890901796
875 | Sandoval |  182.42405890901796
876 | Ehrlich |  182.42405890901796
877 | NBC News |  182.24838453142098
878 | Colts |  182.0669246444806
879 | Hmm |  182.0669246444806
880 | Duffy |  181.61325544803006
881 | Angela Merkel |  181.1374806466914
882 | New Yorks |  180.97159609885708
883 | Goldman Sachs |  180.97159609885705
884 

...

6526 | Tommy Vietor |  23.472438017569875
6527 | Kevin Donohoe |  23.472438017569875
6528 | Rudy Giulianis |  23.472438017569875
6529 | Chicago Blackhawks |  23.472438017569875
6530 | Trump Donald John TrumpIran |  23.472438017569875
6531 | Stefan |  23.472438017569875
6532 | Brett Favre |  23.472438017569875
6533 | Galilee |  23.472438017569875
6534 | Rex Ryan |  23.472438017569875
6535 | Geithners |  23.472438017569875
6536 | Milk |  23.472438017569875
6537 | DeKalb County |  23.472438017569875
6538 | Seuss |  23.472438017569875
6539 | California Democratic |  23.472438017569875
6540 | United States House |  23.472438017569875
6541 | George Pataki |  23.472438017569875
6542 | Muqtada alSadr |  23.472438017569875
6543 | Griffith Park |  23.472438017569875
6544 | Kennedy Space Center |  23.472438017569875
6545 | Shutterstock.co |  23.472438017569875
6546 | BMWs |  23.472438017569875
6547 | Sonia Sotomayor |  23.472438017569875
6548 | Cape |  23.472438017569875
6549 | America and Europe

6719 | Jerry Falwell |  23.472438017569875
6720 | Regnery |  23.472438017569875
6721 | Van Jones |  23.472438017569875
6722 | Elvis Presley |  23.472438017569875
6723 | Emirati |  23.472438017569875
6724 | SATs |  23.472438017569875
6725 | Kiev Ukraine |  23.472438017569875
6726 | Yemen  Dems |  23.472438017569875
6727 | Robert Smith |  23.472438017569875
6728 | Israeli and Palestinian |  23.472438017569875
6729 | Holy Spirit |  23.472438017569875
6730 | Walkers |  23.472438017569875
6731 | Herbert Hoover |  23.472438017569875
6732 | Township |  23.472438017569875
6733 | Write to Charlotte Alter |  23.472438017569875
6734 | Bret Baier |  23.472438017569875
6735 | TEXAS SPORTS NATION |  23.472438017569875
6736 | Sochi Games |  23.472438017569875
6737 | Al Jazeera English |  23.472438017569875
6738 | Gene McCarthy |  23.472438017569875
6739 | Sundar Pichai |  23.472438017569875
6740 | Julian Gill |  23.472438017569875
6741 | Pat Toomey |  23.472438017569875
6742 | PTA |  23.4724380175698

6914 | Sarajevo |  23.472438017569875
6915 | Pride |  23.472438017569875
6916 | Heather |  23.472438017569875
6917 | Westchester |  23.472438017569875
6918 | Twilight |  23.472438017569875
6919 | Mark Jackson |  23.472438017569875
6920 | Plato |  23.472438017569875
6921 | University of California |  23.472438017569875
6922 | Bill Parcells |  23.472438017569875
6923 | LP |  23.472438017569875
6924 | Jared Polis |  23.472438017569875
6925 | Robert Mueller Robert Bob Swan MuellerSchiff Trump |  23.472438017569875
6926 | Quentin Tarantino |  23.472438017569875
6927 | Medical Center |  23.472438017569875
6928 | Los Angelesarea |  23.472438017569875
6929 | East Garfield Park |  23.472438017569875
6930 | RACHEL MARTIN |  23.472438017569875
6931 | Progressive Democrat Congresswomen |  23.472438017569875
6932 | Frances Tiafoe |  23.472438017569875
6933 | Jill Stewart |  23.472438017569875
6934 | SAN FRANCISCOBUnited |  23.472438017569875
6935 | Justice Neil Gorsuch |  23.472438017569875
6936 | 

7110 | National Hockey League |  23.472438017569875
7111 | Department of Agriculture |  23.472438017569875
7112 | Mark Dayton |  23.472438017569875
7113 | Agee |  23.472438017569875
7114 | TOKYO Reuters |  23.472438017569875
7115 | Nuri Kamal alMaliki |  23.472438017569875
7116 | Fairfield |  23.472438017569875
7117 | GMOs |  23.472438017569875
7118 | Kerala |  23.472438017569875
7119 | Alan Blinder |  23.472438017569875
7120 | Battalion |  23.472438017569875
7121 | EditoratLarge at Breitbart News |  23.472438017569875
7122 | Malaysia Airlines Flight |  23.472438017569875
7123 | PMs |  23.472438017569875
7124 | Toronto Raptors |  23.472438017569875
7125 | JPMorgan Chase  Co |  23.472438017569875
7126 | Latest on California |  23.472438017569875
7127 | EW.co |  23.472438017569875
7128 | Abraham Lincolns |  23.472438017569875
7129 | Muammar Gaddafi |  23.472438017569875
7130 | Armstrongs |  23.472438017569875
7131 | Rep. Gabrielle Giffords |  23.472438017569875
7132 | GaryHart |  23.4724

7302 | Robert McNamara |  23.472438017569875
7303 | MBS |  23.472438017569875
7304 | Trump and Clinton |  23.472438017569875
7305 | Hamilton County |  23.472438017569875
7306 | Engels |  23.472438017569875
7307 | itto |  23.472438017569875
7308 | Ted Cruz RTexas |  23.472438017569875
7309 | WMD |  23.472438017569875
7310 | David Simon |  23.472438017569875
7311 | Tim Anderson |  23.472438017569875
7312 | St Petersburg |  23.472438017569875
7313 | VP |  23.472438017569875
7314 | midApril |  23.472438017569875
7315 | Coolidge |  23.472438017569875
7316 | David Dao |  23.472438017569875
7317 | Palo Alto |  23.472438017569875
7318 | Greenwich Village |  23.472438017569875
7319 | Upper West Side |  23.472438017569875
7320 | Marathon |  23.472438017569875
7321 | Byron |  23.472438017569875
7322 | West th Street |  23.472438017569875
7323 | Georgetown University School of Medicine |  23.472438017569875
7324 | Tina Brown |  23.472438017569875
7325 | Wayne Simmonds |  23.472438017569875
7326 | 

In [59]:
processed_sdf.write.parquet(f'features/{VER}')

In [60]:
cv_model.save(f'vocab_model/{VER}')

In [62]:
processed_sdf.printSchema()

root
 |-- authors: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- date_download: string (nullable = true)
 |-- date_modify: string (nullable = true)
 |-- date_publish: string (nullable = true)
 |-- description: string (nullable = true)
 |-- language: string (nullable = true)
 |-- language_guess: string (nullable = true)
 |-- month: long (nullable = true)
 |-- published: string (nullable = true)
 |-- source_domain: string (nullable = true)
 |-- text: string (nullable = true)
 |-- text_or_desc: string (nullable = true)
 |-- title: string (nullable = true)
 |-- url: string (nullable = true)
 |-- year: long (nullable = true)
 |-- id: long (nullable = false)
 |-- clean_text: string (nullable = true)
 |-- finished_sentence: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- finished_token: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- finished_lemma: array (nullable = true)
 |    |-- element: string (conta

In [63]:
processed_sdf.count()

10000